In [1]:
from utz import *
from nj_crashes.paths import ROOT_DIR
from njsp import crashes
from njsp.paths import PROJECTED_CSV
from njsp.ytc import to_ytc
from njsp.ytd import oldest_commit_rundate_since, projected_roy_deaths
from njsp import Ytd

chdir(ROOT_DIR)

In [2]:
ytd = Ytd()
prv_rundate = ytd.prv_rundate
prv_rundate

Searching for oldest commit with rundate ≥2024-12-21


Found rundate 2024-12-20 10:00:03 < 2024-12-21 at commit ccb16101; returning commit 9623302


'2024-12-21 10:00:04'

In [3]:
prv_ytd_ratio = ytd.cur_ytd_deaths / ytd.prv_ytd_deaths
pct_change = (prv_ytd_ratio - 1) * 100

print(f'As of {ytd.prv_rundate}, NJSP was reporting {ytd.prv_ytd_total} YTD deaths')
print(f'Current YTD Deaths ({ytd.rundate}): {ytd.cur_ytd_deaths}')
print(f'Previous year YTD Deaths (adjusted; {ytd.prv_rundate}): {ytd.prv_ytd_deaths}')
print(f'Projected {ytd.cur_year} total: {ytd.projected_year_total}')
print(f'{pct_change:.1f}% change')

As of 2024-12-21 10:00:04, NJSP was reporting 674 YTD deaths
Current YTD Deaths (2025-12-21 10:00:59-05:00): 552
Previous year YTD Deaths (adjusted; 2024-12-21 10:00:04): 673.9462247840919
Projected 2025 total: 560.2873535006632
-18.1% change


5434ab28d8c7f891810bd95bf5bfae9ddd3d5e2c: FAUQStats cache miss: 2024, Sat Dec 21 10:00:04 EST 2024


Find the oldest commit with rundate less than 1 year ago, load crashes from that time:

In [4]:
prv_crashes = ytd.prv_ytd_crashes
prv_crashes

,CCODE,CNAME,MCODE,MNAME,HIGHWAY,LOCATION,FATALITIES,FATAL_D,FATAL_P,FATAL_T,FATAL_B,INJURIES,STREET,dt
ACCID,,,,,,,,,,,,,,
12918,12,Middlesex,1218,Plainsboro Twsp,1,State Highway 1,1.0,0.0,0.0,1.0,0.0,4.0,NaN,2024-01-01 01:56:00-05:00
12923,11,Mercer,1111,Trenton,129,State Highway 129,1.0,1.0,0.0,0.0,0.0,NaN,NaN,2024-01-03 12:14:00-05:00
12932,01,Atlantic,0112,Hamilton Twsp,446,Atlantic City Expressway MP 20.5,1.0,1.0,0.0,0.0,0.0,0.0,NaN,2024-01-05 09:14:00-05:00
12933,02,Bergen,0248,Ramsey Boro,NaN,East Oak St,1.0,1.0,0.0,0.0,0.0,0.0,East Oak St,2024-01-05 14:12:00-05:00
13362,07,Essex,0701,Belleville Town,647,County 647 MP .76,1.0,0.0,0.0,1.0,0.0,NaN,NaN,2024-01-05 18:08:00-05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13655,14,Morris,1427,Mount Olive Twsp,206,State Highway 206 MP 92,1.0,1.0,0.0,0.0,0.0,0.0,NaN,2024-12-18 05:11:00-05:00
13657,08,Gloucester,0810,Mantua Twsp,45,State Highway 45,1.0,1.0,0.0,0.0,0.0,NaN,NaN,2024-12-19 10:48:00-05:00
13659,06,Cumberland,0605,Fairfield Twsp,658,County 658,1.0,0.0,0.0,1.0,0.0,NaN,NaN,2024-12-19 18:35:00-05:00


Load previous year YTD and final counts, and current year YTD counts:

In [5]:
prv_year = ytd.prv_year
cur_year = ytd.cur_year

prv_ytc = to_ytc(prv_crashes)
prv_ytd = prv_ytc.loc[prv_year]

prv_end = to_ytc(ytd.prv_end_crashes).loc[prv_year]

cur_ytc = to_ytc(ytd.cur_ytd_crashes)
cur_ytd = cur_ytc.loc[cur_year]

cur_ytd

,driver,passenger,pedestrian,cyclist,crashes
county,,,,,
Atlantic,17,4,12,0,32
Bergen,22,2,7,0,31
Burlington,23,5,8,1,35
Camden,20,3,15,2,39
Cape May,10,4,0,0,12
Cumberland,10,2,8,0,19
Essex,10,5,24,1,40
Gloucester,18,6,3,1,25
Hudson,8,1,8,2,17


Compute fraction of current year that has elapsed (year-to-date; "YTD") and still remains (rest-of-year; "ROY"):

In [6]:
cur_ytd_frac = ytd.cur_year_frac
cur_roy_frac = 1 - cur_ytd_frac
print('%.1f%% through the year, %.1f%% remaining' % (ytd.cur_year_frac * 100, ytd.cur_roy_frac * 100))

97.1% through the year, 2.9% remaining


Combine previous year YTD / end with current year YTD:

In [7]:
def melt(df, name):
    return (
        df
        .melt(ignore_index=False, var_name='type')
        .set_index('type', append=True)
        .value
        .rename(name)
    )

z = sxs(
    melt(prv_ytd, 'prv_ytd'),
    melt(prv_end, 'prv_end'),
    melt(cur_ytd, 'cur_ytd'),
).fillna(0).astype(int)
z

,,prv_ytd,prv_end,cur_ytd
county,type,,,
Atlantic,driver,31,32,17
Bergen,driver,15,14,22
Burlington,driver,31,31,23
Camden,driver,18,19,20
Cape May,driver,8,8,10
...,...,...,...,...
Salem,crashes,13,14,18
Somerset,crashes,19,18,12
Sussex,crashes,13,12,11


In [8]:
def project_roy(r):
    return int(round(projected_roy_deaths(r.prv_ytd, r.prv_end, r.cur_ytd, cur_ytd_frac)))

roy = z.apply(project_roy, axis=1).rename('roy')
projected = (
    (z.cur_ytd + roy)
    .rename('projected')
    .reset_index(level=1)
    .pivot(columns='type', values='projected')
)
projected

type,crashes,cyclist,driver,passenger,pedestrian
county,,,,,
Atlantic,33,0,18,4,13
Bergen,32,0,21,2,8
Burlington,36,1,23,5,9
Camden,40,2,21,4,14
Cape May,13,0,10,4,0
Cumberland,18,0,9,2,7
Essex,41,1,11,5,23
Gloucester,25,1,18,6,3
Hudson,18,0,8,1,11


In [9]:
projected.to_csv(PROJECTED_CSV)